In [10]:
# Importar librerías
import agentpy as ap
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
import random

In [141]:
Movimientos = [[0,1],[1,0],[0,-1],[-1,0],[1,1],[1,-1],[-1,1],[-1,-1]]

class ModeloRobots(ap.Model):

    def setup(self):

        # Crear agentes (Robots)
        self.robots = self.agents = ap.AgentList(self, self.p['agents'])

        # Crear agentes (Celdas)
        self.celdas = self.agents = ap.AgentList(self,int((self.p['PorcentajeCeldas'] * self.p.sizeM * self.p.sizeN)))

        # Crear grid (Piso)
        self.piso = ap.Grid(self, (self.p.sizeM,self.p.sizeN), track_empty=True)
        self.piso.add_agents(self.celdas, random=True, empty=True)
        self.piso.add_agents(self.robots, [(1,1)] * self.p['agents'], random = False, empty = False)

        # Variable dinámica para el estado de cada una de las celdas
        # Condition 0: Robot, 1: Celdas Sucias, 2: Limpio
        self.robots.condition = 0
        self.celdas.condition = 1

    def step(self):

        # Buscar celdas sucias cercanas a la limpia
        for robot in self.robots:
            for neighbor in self.piso.neighbors(robot):
                if neighbor.condition == 1: #Si la celda está sucia
                    neighbor.condition = 2 # Se limpia la celda sucia
                    break
            else: #Si la celda está limpiándose o está limpia
                self.piso.move_by(robot, random.choice(Movimientos)) #Se mueve en cualquiera de las 8 direcciones

        celdasSucias = self.celdas.select(self.celdas.condition == 1)
        # Detener la simulación si todas las celdas están limpias
        if len(celdasSucias) == 0:
            self.stop()

    def end(self):

        # Recabar datos de la simulación
        celdas_Limpias = len(self.agents.select(self.agents.condition == 2))
        self.report('Porcentaje de celdas limpias: ',
                    celdas_Limpias / len(self.celdas))

In [142]:
# Definir parámetros iniciales

parameters = {
    'PorcentajeCeldas': 1, # % de celdas en el piso
    'sizeM': 10, # Medidas del grid
    'sizeN': 10,
    'steps': 100,
    'agents': 5, # Número de robots limpiadores
}

In [143]:
# Animación para mostrar resultados

def animation_plot(model, ax):
    attr_grid = model.piso.attr_grid('condition')
    color_dict = {0:'Blue', 1:'#6E4912', 2:'White', None:'#Gray'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulación de limpieza de un piso\n"
                 f"Tiempo: {model.t}, Celdas por limpiar: "
                 f"{len(model.agents.select(model.agents.condition == 1))}", color = "white")

fig, ax = plt.subplots()
model = ModeloRobots(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))